# Patient Reviewer

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from JupyterReviewer.Reviewers.PatientReviewer import PatientReviewer
from JupyterReviewer.ReviewData import ReviewDataAnnotation

import pandas as pd

## Edit the code below according to the commented directions

In [8]:
reviewer = PatientReviewer()

reviewer.set_review_data(
    # Enter the file path where you would like your pkl file of your review data to be stored
    review_data_fn='/Users/svanseters/Broad/JupyterReviewer/review.pkl',
    # Breifly describe the source of your data and its purpose
    description='first review',
    # Enter the file path to your data to be reviewed
    df=pd.read_csv(f'/Users/svanseters/Broad/JupyterReviewer/example_notebooks/example_data/test_data_for_reviewer.txt', sep='\t')
)

reviewer.set_review_app(test_param='testing param kwargs')
reviewer.run(mode='external', port=8088)

Loading existing review session /Users/svanseters/Broad/JupyterReviewer/review.pkl...
Dash app running on http://0.0.0.0:8088/


## Add your own components below 